In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../raw_data/raw_en.csv')

df.info()

/tmp/ipykernel_6922/1251664015.py:6: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/raw_en.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457074 entries, 0 to 457073
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            457074 non-null  object 
 1   date            351179 non-null  object 
 2   recommendation  350678 non-null  object 
 3   review          457074 non-null  object 
 4   playing_hours   350678 non-null  float64
 5   source          457074 non-null  object 
 6   stars           106396 non-null  float64
 7   language        457074 non-null  object 
dtypes: float64(2), object(6)
memory usage: 27.9+ MB


In [3]:
df['language'].value_counts()

en    457074
Name: language, dtype: int64

In [5]:
df1 = df.head(10000)
# creating small sample to train with

In [15]:
from customerchurn.pre_pipeline import create_pre_pipe
from customerchurn.pre_pipeline import series_prepro
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer



pipe = create_pre_pipe()

pipe.fit(df1)

df1['review_clean'] = pipe.transform(df1)[:,0]

/tmp/ipykernel_6922/3206781970.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['review_clean'] = pipe.transform(df1)[:,0]


In [14]:
df2[:,0]

array(['gt played german reichgt declare war belgiumgt cant break belgium go francegt capitulate france order get belgiumgt get true blitzkrieg achievementthis game dad',
       'good game although bit overpriced opinion id prefer playing game mod historical accuracy although vanilla version good aswell',
       'review wrote one probably serious one wrote starter community game suck like every online game dont wan na talk theyre jerk wan na say suck camped blah blah blah rare get nice compliment people gamebut dont talk community either unlike online game one doesnt mic system dont worry hearing twelve year old cry baby raging nothing time talk people game beginning match start survivor end match killer survivor end chat one killer talk everyone match though mostly say gg good gameas survivor basically play every man sneaky say screw player try good team mate rescue get hook someone jerk chat match know save get hooked mostly play killer find fun playing survivor someone got ta play k

In [ ]:
#when I run the